In [19]:
#importing set of libraries
import pandas as pd 
import numpy as np

import matplotlib.pyplot as plt 

import re 
#Package for splitting the data
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import LabelEncoder 
#Tokenization
from keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from keras.models import Sequential 
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D 
from keras.utils.np_utils import to_categorical

In [20]:
from google.colab import drive 
drive.mount('/content/gdrive') 

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [21]:
import pandas as pd

# Loading the dataset as a Pandas DataFrame
dataset = pd.read_csv('/content/gdrive/My Drive/Sentiment.csv')

# Selecting only the necessary columns 'text' and 'sentiment'
mask = dataset.columns.isin(['text', 'sentiment'])
data = dataset.loc[:, mask]



In [22]:
# Keeping only necessary columns
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

<ipython-input-22-4d9282c66e4b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['text'].apply(lambda x: x.lower())
<ipython-input-22-4d9282c66e4b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))


In [23]:
#Removing Re tweets
for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ') 

In [24]:
#Maximum words is 2000 to tokenize the sentence
max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' ') 
tokenizer.fit_on_texts(data['text'].values) 
X = tokenizer.texts_to_sequences(data['text'].values) 


In [25]:
#Padding the feature matrix
X = pad_sequences(X) 
embed_dim = 128 
#Long short-term memory (LSTM) layer 
lstm_out = 196 

In [26]:
def createmodel():
    model = Sequential() #Sequential Neural Network
    #input dimension - 2000 Neurons, output dimension-128 Neurons
    model.add(Embedding(max_features, embed_dim,input_length = X.shape[1])) 
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2)) 
    model.add(Dense(3,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy']) #Compiling the model
    return model


In [27]:
#Applying label Encoding on the label matrix
labelencoder = LabelEncoder() 
integer_encoded = labelencoder.fit_transform(data['sentiment']) 
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42) 

In [28]:
batch_size = 32 #Batch size 32
model = createmodel() 
model.fit(X_train, Y_train, epochs = 1, batch_size=batch_size, verbose = 2) #verbose the higher, the more messages
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size) #evaluating the model
print(score)
print(acc)

291/291 - 53s - loss: 0.8225 - accuracy: 0.6424 - 53s/epoch - 182ms/step
144/144 - 2s - loss: 0.7563 - accuracy: 0.6732 - 2s/epoch - 11ms/step
0.7562976479530334
0.6732197403907776


In [29]:
print(model.metrics_names) #metrics of the model

['loss', 'accuracy']


In [ ]:
#1)Save the model and use the saved model to predict on new text data (ex, “A lot of good things are
#happening. We are respected again throughout the world, and that's a great thing.@realDonaldTrump”)

In [30]:
#Saving the model
model.save('sentimentAnalysis.h5') 

In [31]:
#Importing the package for importing the saved model
from keras.models import load_model 
model= load_model('sentimentAnalysis.h5') #loading the saved model

In [32]:
print(integer_encoded)
print(data['sentiment'])

[1 2 1 ... 2 0 2]
0         Neutral
1        Positive
2         Neutral
3        Positive
4        Positive
           ...   
13866    Negative
13867    Positive
13868    Positive
13869    Negative
13870    Positive
Name: sentiment, Length: 13871, dtype: object


In [33]:
# Predicting on the text data
sentence = ['A lot of good things are happening. We are respected again throughout the world, and that is a great thing.@realDonaldTrump']
sentence = tokenizer.texts_to_sequences(sentence) # Tokenizing the sentence
sentence = pad_sequences(sentence, maxlen=28, dtype='int32', value=0) # Padding the sentence
sentiment_probs = model.predict(sentence, batch_size=1, verbose=2)[0] # Predicting the sentence text
sentiment = np.argmax(sentiment_probs)

print(sentiment_probs)
if sentiment == 0:
    print("Neutral")
elif sentiment < 0:
    print("Negative")
elif sentiment > 0:
    print("Positive")
else:
    print("Cannot be determined")


1/1 - 0s - 236ms/epoch - 236ms/step
[0.6077131  0.1597905  0.23249641]
Neutral


In [ ]:
#2. Apply GridSearchCV on the source code provided in the class

In [34]:
#importing Keras classifier
from keras.wrappers.scikit_learn import KerasClassifier
#importing Grid search CV 
from sklearn.model_selection import GridSearchCV 
#initiating model to test performance by applying multiple hyper parameters
model = KerasClassifier(build_fn=createmodel,verbose=2) 
#batch_size
batch_size= [10, 20, 40] 
# no. of epochs
epochs = [1, 2] 
#dictionaries
param_grid= {'batch_size':batch_size, 'epochs':epochs}
grid  = GridSearchCV(estimator=model, param_grid=param_grid) 
grid_result= grid.fit(X_train,Y_train) #Fitting the model
#summarizing the  results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_)) #best score, best hyper parameters

<ipython-input-34-b32174a39385>:6: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=createmodel,verbose=2)


744/744 - 113s - loss: 0.8280 - accuracy: 0.6437 - 113s/epoch - 152ms/step
186/186 - 2s - loss: 0.7708 - accuracy: 0.6455 - 2s/epoch - 12ms/step


744/744 - 114s - loss: 0.8175 - accuracy: 0.6501 - 114s/epoch - 153ms/step
186/186 - 2s - loss: 0.7448 - accuracy: 0.6810 - 2s/epoch - 12ms/step


744/744 - 113s - loss: 0.8236 - accuracy: 0.6465 - 113s/epoch - 152ms/step
186/186 - 2s - loss: 0.7618 - accuracy: 0.6896 - 2s/epoch - 12ms/step


744/744 - 114s - loss: 0.8278 - accuracy: 0.6408 - 114s/epoch - 153ms/step
186/186 - 3s - loss: 0.7695 - accuracy: 0.6695 - 3s/epoch - 15ms/step


...

Epoch 1/2
186/186 - 40s - loss: 0.8398 - accuracy: 0.6408 - 40s/epoch - 214ms/step
Epoch 2/2
186/186 - 27s - loss: 0.6931 - accuracy: 0.7045 - 27s/epoch - 144ms/step
47/47 - 1s - loss: 0.7374 - accuracy: 0.6826 - 798ms/epoch - 17ms/step


Epoch 1/2
186/186 - 40s - loss: 0.8472 - accuracy: 0.6330 - 40s/epoch - 214ms/step
Epoch 2/2
186/186 - 25s - loss: 0.6872 - accuracy: 0.7062 - 25s/epoch - 132ms/step
47/47 - 1s - loss: 0.7580 - accuracy: 0.6848 - 1s/epoch - 23ms/step


Epoch 1/2
186/186 - 37s - loss: 0.8412 - accuracy: 0.6399 - 37s/epoch - 197ms/step
Epoch 2/2
186/186 - 26s - loss: 0.6808 - accuracy: 0.7085 - 26s/epoch - 139ms/step
47/47 - 1s - loss: 0.7362 - accuracy: 0.6835 - 738ms/epoch - 16ms/step


Epoch 1/2
186/186 - 36s - loss: 0.8337 - accuracy: 0.6363 - 36s/epoch - 193ms/step
Epoch 2/2
186/186 - 28s - loss: 0.6732 - accuracy: 0.7138 - 28s/epoch - 152ms/step
47/47 - 1s - loss: 0.7800 - accuracy: 0.6728 - 725ms/epoch - 15ms/step


Epoch 1/2
233/233 - 47s - loss: 0.8212 - accuracy: 0.6427 - 47s/epoch - 201ms/step
Epoch 2/2
233/233 - 31s - loss: 0.6823 - accuracy: 0.7092 - 31s/epoch - 135ms/step
Best: 0.680834 using {'batch_size': 40, 'epochs': 2}
